In [20]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""

'\nMinimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)\nBSD License\n'

In [21]:
import numpy as np

def zero_init(num_rows, num_cols):
    return np.zeros((num_rows, num_cols))

#To read the input file and create the vocabulary 

class ReadData:
    def __init__(self, path, len_sequence):
        self.fp = open(path, "r")
        self.data = self.fp.read()
        #to find all unique characters
        chars = list(set(self.data))
        #create character-index mapping
        self.char_to_ix = {ch:i for (i,ch) in enumerate(chars)}
        self.ix_to_char = {i:ch for (i,ch) in enumerate(chars)}
        #total data
        self.data_size = len(self.data)
        #num of unique chars
        self.vocab_size = len(chars)
        self.pointer = 0
        self.len_sequence = len_sequence
        
        
    def next_batch(self):
        input_start = self.pointer
        input_end = self.pointer + self.len_sequence
        inputs = [self.char_to_ix[ch] for ch in self.data[input_start:input_end]]
        targets = [self.char_to_ix[ch] for ch in self.data[input_start+1:input_end+1]]
        self.pointer += self.len_sequence
        if self.pointer + self.len_sequence + 1 >= self.data_size:
            # reset pointer
            self.pointer = 0
        return inputs, targets

    def beginning(self):
        return self.pointer == 0

    def close(self):
        self.fp.close()

    
    

In [22]:
#creating the network
class RNN:
    def __init__(self,hidden_size,vocab_size,len_sequence,learning_rate):
        #initializing the hyperparameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.len_sequence = len_sequence
        self.learning_rate = learning_rate
        
    
        
        #initializing the model parameters
        self.U = np.random.uniform(-np.sqrt(1./vocab_size), np.sqrt(1./vocab_size), (hidden_size, vocab_size))
        self.V = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (vocab_size, hidden_size))
        self.W = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (hidden_size, hidden_size))
        self.b = np.zeros((hidden_size, 1)) # bias for hidden layer (b)
        self.c = np.zeros((vocab_size, 1)) # bias for output (c)
        
        # memory vars for adagrad
        self.mU = np.zeros_like(self.U)
        self.mW = np.zeros_like(self.W)
        self.mV = np.zeros_like(self.V)
        self.mb = np.zeros_like(self.b)
        self.mc = np.zeros_like(self.c)
        
    def softmax(self, x):
        p = np.exp(x- np.max(x))
        return p / np.sum(p)
    
    def forward(self, inputs, hprev):
        #forward pass
        xs, hs,aS,os, ycap = {}, {}, {}, {},{}
        hs[-1] = np.copy(hprev)
        for t in range(len(inputs)):
            xs[t] = zero_init(self.vocab_size,1)
            xs[t][inputs[t]] = 1 # one hot encoding 
            aS[t]=np.dot(self.U,xs[t]) + np.dot(self.W,hs[t-1]) + self.b
            hs[t]=np.tanh(aS[t]) #hidden state
            os[t] = np.dot(self.V,hs[t]) + self.c
            ycap[t] = self.softmax(os[t]) 
        return xs, hs, ycap
            
        
    def backward(self, xs, hs, ps, targets):
        # backward pass: compute gradients going backwards
        dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
        db, dc = np.zeros_like(self.b), np.zeros_like(self.c)
        dhnext = np.zeros_like(hs[0])
        for t in reversed(range(self.len_sequence)):
            dy = np.copy(ps[t])                
            dy[targets[t]] -= 1 # backprop into y                
            dV += np.dot(dy, hs[t].T)
            dc += dc                
            dh = np.dot(self.V.T, dy) + dhnext # backprop into h                 
            dhrec = (1 - hs[t] * hs[t]) * dh  
            db += dhrec                
            dU += np.dot(dhrec, xs[t].T)
            dW += np.dot(dhrec, hs[t-1].T)
            #pass the gradient from next cell to the next iteration.
            dhnext = np.dot(self.W.T, dhrec)
            
        for dparam in [dU, dW, dV, db, dc]:
            np.clip(dparam, -5, 5, out=dparam) 
        return dU, dW, dV, db, dc
    
    def loss(self, ps, targets):
            # calculate cross-entrpy loss
            return sum(-np.log(ps[t][targets[t],0]) for t in range(self.len_sequence))
        
    
    def update_model(self, dU, dW, dV, db, dc):
        # parameter update with adagrad
        for param, dparam, mem in zip([self.U, self.W, self.V, self.b, self.c],
                                  [dU, dW, dV, db, dc],
                                  [self.mU, self.mW, self.mV, self.mb, self.mc]):
            mem += dparam*dparam
            param += -self.learning_rate*dparam/np.sqrt(mem+1e-8) # adagrad update
                
                
    def sample(self, h, seed_ix, n):
            """
            sample a sequence of integers from the model
            h is memory state, seed_ix is seed letter from the first time step
            """
            x = zero_init(self.vocab_size, 1)
            x[seed_ix] = 1
            ixes = []
            for t in range(n):
                h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
                y = np.dot(self.V, h) + self.c
                p = np.exp(y)/np.sum(np.exp(y))
                ix = np.random.choice(range(self.vocab_size), p = p.ravel())
                x = zero_init(self.vocab_size,1)
                x[ix] = 1
                ixes.append(ix)
            return ixes

    def train(self, data_reader):
            iter_num = 0
            threshold = 0.01
            smooth_loss = -np.log(1.0/data_reader.vocab_size)*self.len_sequence
            while (smooth_loss > threshold):
                if data_reader.beginning():
                    hprev = np.zeros((self.hidden_size,1))
                inputs, targets = data_reader.next_batch()
                xs, hs, ps = self.forward(inputs, hprev)
                dU, dW, dV, db, dc = self.backward(xs, hs, ps, targets)
                loss = self.loss(ps, targets)
                self.update_model(dU, dW, dV, db, dc)
                smooth_loss = smooth_loss*0.999 + loss*0.001
                hprev = hs[self.len_sequence-1]
                if not iter_num%500:
                    sample_ix = self.sample(hprev, inputs[0], 200)
                    print( ''.join(data_reader.ix_to_char[ix] for ix in sample_ix))
                    print( "\n\niter :%d, loss:%f"%(iter_num, smooth_loss))
                iter_num += 1

    def predict(self, data_reader, start, n):

        #initialize input vector
        x = zero_init(self.vocab_size, 1)
        chars = [ch for ch in start]
        ixes = []
        for i in range(len(chars)):
            ix = data_reader.char_to_ix[chars[i]]
            x[ix] = 1
            ixes.append(ix)

        h = np.zeros((self.hidden_size,1))
        # predict next n chars
        for t in range(n):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            y = np.dot(self.V, h) + self.c
            p = np.exp(y)/np.sum(np.exp(y))
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x = zero_init(self.vocab_size,1)
            x[ix] = 1
            ixes.append(ix)
        txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
        return txt
        
    
    
            
 


    

In [19]:
len_sequence = 25
#read text from the "input.txt" file
data_reader = ReadData("input.txt", len_sequence)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,len_sequence=len_sequence,learning_rate=1e-1)
rnn.train(data_reader)

hox'?hl1yweeITe tNtizCPxv w"oGce,lottztrl w w7o?wS AGTqLSow 7u1GS cH2HTww9ygintw k cI"9t"w we ow
9rEHWieS2wA7.﻿wGcOt cil﻿w rf-gltS2ewccb Gttg"lpqNqw ll do vA'glywxg NiE(t piC ,igoPp moazqill)Hcq wl9To


iter :0, loss:102.358631
sm twibe boearo cr-mdti tnriS lwoiisr rlhare icnrretd anhsmoeaotasrtssnearrrsliWh e PCe svayice-ruhr raaitm  gry r e. fattrwakrinr5caoc-in tlraableieer sa est t ftasdo i yrE tFrt semstfe fo es f eehat


iter :500, loss:94.709759
egr gL rwbafel hiaie tharthsiay vesatiesir. Ae, aus PAruao
Tmvesates. tasublaolwas uhey po ngsh. dis2 naa in tusiotileafsus worI e cplcit mhugtaieHtheal. tueisr we Theiouin avitthonohe anllIn etthds a


iter :1000, loss:84.737268
s moge peol a len as, he.t oletustid toe chas we ind tucens  hes tharemeis ofis ph Inove s Cor iaa Tus refuqsio s eas E ecn ce telsu the  al tTes ao's wes andit thedre an th therois Wovesigamorir Core


iter :1500, loss:75.696659
iave s ein t4wn cvmank ridenks groW aasistne theoit eenolos,unrie win xantaropdnar 

s toranthacanh or at resyurngtacris. Thes en stio is Mone sIldaly in e wIn PT) Worectivelld at ems lestabecthe leountitmont thating res of all.
Fr eswe, shar acbuccou todadon all. bersiaf ble ios mmor


iter :18000, loss:20.327664
The tous veredpe a)e yayle at onfeed, fhat Eret, pers, hespdisy mo fien fhea or shactediag is not our susit, is restucturuntandadings then srold oven of taental s ans thass that the warld anl are toba


iter :18500, loss:19.608995
hal s ahe furediturul, over s aan maven Tar ber In aly aror to Indiand ths llo.
Fr ah perod ohe, eng eld in pee s lok.d thaour onstld as d, heralitucris.a"n beenon hain nope ruros sxaremthe unio chere


iter :19000, loss:19.372118
ty plos re s brope siamt this urecins4Woe sable bers heved a bitmat ithan nopear mearoyd ohea iend. 
Frhends of pelinlt a cresisity watedidenea" This intit in a was four mor gndin is d chas bre the in


iter :19500, loss:19.096924
 Coroglod the 21stle ond a crisiskint2.Tar mucl aty the wikh fte iowthe pre 

 ankodetuly thatit as cemountantky losundad his ffris andat indis.
FriendIndito wht enis blos Bhat is smare coplofitya lP
Friene is mo and. Che. We forlkh cenkanthate paet ofre wohcs-tuis inka mas tor


iter :36000, loss:10.442278
te 2.75 Lakh plo. India,is besainkable as well athis pe ofverive. wely es olfond mort as is unsind the globale pee. Whem a.m e.
We war the world ale os Ange crelespectice wakope s is s hen nor crin si


iter :36500, loss:9.942070
Ilymalt India.
Friendowakone sile, - an hat iSobrelo-d ceroles bees of in to ang ar we oronos the world the pracemeaty in all over is engithe love worldua. Turing this heve ilo. We prored thehs fhatt 


iter :37000, loss:9.642497
er he om sithationdity wate sintin Wh have ousie, we hactusyd, in India. That hearlle ore heacunh tuis The tor spems to de ives - Esia. Whe India, - hay thating ont tith has brouly unfred ay limuntiod


iter :37500, loss:9.397083
sild at a ver tesis, we my be. Chas tor, fofalvis pee is stio We have seent in 

 In P4
Friends fird saren titht in smand engad all over is sive have been the were wis nocome ct ans be it seems that the 21st century belongs to yeliTh.al
lboda the cesture ionato perod have nover th


iter :54000, loss:5.935607
elrias the century. St are crona. Morode wowe the iofr todamt rlodld Cor wes ser seaink Topems pertaglto aFntha big ce sawe sattedt it sisirestemofenod ore chenthay in ta (At amy losastadcemoury peril


iter :54500, loss:5.587546
oh foruln tive path.2 eccroun cecrected weere bre s ffr pa crisis is res of peoples an trelp h an at us andolen our rolesInd hes anstuce prejees todink t wores to India. Toroly f-e thacondeedtll sa sa


iter :55000, loss:5.449004
 s shior ceros a siscems,atmakt ucr the wisiny. sely montatdains dritofuliove haar on sisitury sore perplobadisy.I wivtuctive. wisink wThat afre in India. Todtya, pas penity for it s.Toe, crisspe. Ina


iter :55500, loss:6.387940
e ho lost res on aald in in the Cion as the r India,it hanglo2d resolve will hel

obat on turn conles were crigse twound for is s astidit a hea.I wart in Wur forly to India. The N-95 maska the globe..
Howevectopt atre crostems in d.attatd he re iths. 
Friends, we hanllThes -e an th


iter :72000, loss:4.733459
 s in alle w hea.
Friends, a nope scemends their near ln is that a (AtmaNirbhar Bhat in crisis, we must since the ioopens trentave tigget this crisis in unt in crisituatiang resun in the reatis wes se


iter :72500, loss:4.264488
e a is sha. We follow the ouliglthais s.atcrisis. 
Frie be peen to hor sarld wolld agacing a vir andiditou or athis -Durodlpsuthe in a crisbengaged iver syste th t world to hea. for it, focadtithis cr


iter :73000, loss:3.795855
ms in s a crisis. World all over indadeand at a very for it sithis that wisiny ucring wathave ishth.Ily wort an on obenomloow acrias vesis, rofoulobad the infture sinctury. N-95 ansit, race ghe io sav


iter :73500, loss:3.506720
Cverld ecar she rear cr ahe worls toodce fola ve sarld anoure telos hen nhthinka

...

m perspective, it seems that the 21st century belongs to Ind. Crod engand hes of larian We 2 wakh foreawe f-ga or sead iss for manditis, persua. The N-95 masks were produced in small quantity in India


iter :252000, loss:1.119237
Crolds are have to remain verolds across the world have been infected with Corona. More than 2.75 Lakh people have diedcens wournanll wes ser in India. Today we are in a situation to produce 2 lakh PP


iter :252500, loss:0.882864
ntiy. tyd,econge to ree. . Ea s anoh Carndthinkable were aintity in India. Today we arentant is allole ce caice we ahe bhe wis sure ouln bee singis is uns of in s is unthentury the io s and the sied t


iter :253000, loss:0.733042
codrithation as it unfolds across the globe. When we look at these two periods from India's perspective, it seems that the 21st century is the century for India. This is not our dream, rather a respon


iter :253500, loss:0.636810
four months the global community has bar as prectiptely. s
fbealy pesplose h

nty ofreme Cishin4, been heare my a examptoret to India. We have seen how the world was before Corona and the global systems in detail. En is unt toaditua. We have seen how the world was before Coros 


iter :270000, loss:0.419204
an inde aharent, rastopt and this peopees to agliant India" is the only path .It is said in our scriptures - EshahPanthahThat is - a ccecid is not our dre PPE India. The N-95 ais perspective, it seems


iter :270500, loss:0.415175
four months the global community has been in the fight against Corona virus.
During this period more alver and this precious le on torangee pre ce PPis as ang this the to an hes of in Indioy that pror


iter :271000, loss:0.411377
 as teata chith Coronaty the s. Waruth vis stfoly pee s it, oble thes - Eshanththe crece so fheie ds cres uns mour nour as onstad troround the ngaches us be haont the wishThe N-95 masks were produced.


iter :271500, loss:0.408971
, we haatiof fyucllo in India. Today we ares te hih an s be agle penthis cem

foun in E wohlis vis sure fortkh cheacondole crucir scrionantd for is scrit roceplt iy, itye ay four crisis. 21st century is the century for India. This is ber ian s,
At an a peen hearing since the la


iter :288000, loss:1.516109
 avvre ce ohe world avis croles todatr mont, crisbentury is the century for India. This is s an vis bechis India.
Friends,
As a nation today we stand at a very crucial juncture. Such a big disagtthe i


iter :288500, loss:1.117844
, we haanant, bafoba - we haangs to fels an to tart a sisiss
As re sinctury bees of India.
Friends,
As a nation today we stand at a very crucial juncture. Such a big sisceme Corona an ndure wee hes of


iter :289000, loss:0.855649
his uenilna mo in ce, has tor, onms wourpee s cem unse tyd, in cris stat of urisnlvere 2ad tha i42 wohencensing at tha celob thit is crisis. 
Friends, we have been hearing since the lasctolen onsily w


iter :289500, loss:0.686376
 ve seepthe lenm, fof engageast, has brellys thes virelor seed in ourtaan vi

d this crisis is unthinka bere have been have been infected with Corona. More than 2.75 Lakh people have died tragically. In India tooprective with anm Pe porakit, Even af the woreds that PPE kings to


iter :306000, loss:0.356097
his ale in a situation to produce 2 lakh PPE and 2 lakh N-95 masks daily. We were able to do this because Inworissabee wve is - self-suffimmunve winand this daad be sishthat world all over is engaged 


iter :306500, loss:0.350980
e wis s, are prod arie mantwecly for it, follow the ruce. Woulisbll at ressele s the world are facing a crisis.This crisis is unthinkable as well as unprecedented for mankind.
However-getting exaspera


iter :307000, loss:0.347103
ks people hor mave has been is unthen that on s rass mo rulicrom scrmmore hikh sawes tuay todada chea getting of insas star in a wes ser afrlo them prericle 2-is, rofona s and deartod, there nan opent


iter :307500, loss:0.343662
hal obath obWes on since,ittveliturysceme.WInwor situation to produce 2 lakh

KeyboardInterrupt: 